In [95]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

from src.utils import evaluate_retriever
from src.Retriever import ThresholdRetriever
from src.utils import RetrieverMetrics, save_retriever_trial_log, prepare_thresholdretriever_configs, load_benchmarks_df

In [96]:
# !!! TO CHANGE !!!
TRIAL = 24
BENCHMARKS_MAXSIZE = 10000
BENCHMARKS_INFO = {'mtssquad': {'db': 'v2', 'table': 'v1'}}

RETRIEVER_PARAMS = {
    "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
    "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
    "model_kwargs": {'device':'cuda'},
    "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
    "params": {'fetch_k': 50, 'threshold': 0.23, 'max_k': 3}
}

ADDITIONAL_PARAMS = {
    'topk_score_list': 3
}
# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

In [97]:
retrivers_config, benchmarks_path = prepare_thresholdretriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
metrics = RetrieverMetrics()

In [98]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

In [99]:
RETRIEVERS = {name: ThresholdRetriever(retrivers_config[name]) for name in benchmarks_df.keys()}

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/rag_project/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [100]:
retriever_scores, retriever_cache, _ = evaluate_retriever(benchmarks_df, RETRIEVERS, metrics, topk_score_list=ADDITIONAL_PARAMS['topk_score_list'])

mtssquad


  0%|          | 0/10000 [00:00<?, ?it/s]/trinity/home/team06/workspace/mikhail_workspace/rag_project/src/utils/evaluation_metrics.py:39: RuntimeWarning: invalid value encountered in scalar divide
  return (2 * self.precision(pred_cands, gold_cands, k) * self.recall(pred_cands, gold_cands, k)) / (self.precision(pred_cands, gold_cands, k) + self.recall(pred_cands, gold_cands, k))
 28%|██▊       | 2784/10000 [00:43<01:53, 63.64it/s, MRR=0.805, mAP=0.805, Recall=0.867, Precision=0.289, F1=0.434]


KeyboardInterrupt: 

In [94]:
save_retriever_trial_log(SAVE_LOGDIR, retriever_scores, SAVE_HYPERPARAMS, SAVE_RETRIEVERCACHE,
                         retriever_cache, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE, RETRIEVER_PARAMS, ADDITIONAL_PARAMS)